# Домашнее задание
## Выполнил студент второй подгруппы
## Ворошилов Владислав
## Группа: ПИ20-3


### Подключаем библиотеку 

In [2]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_
from sqlalchemy.sql import func

In [3]:
# Путь до базы данных на вашем пк
path_to_db = "sqlite:////Users/vladislavvorosilov/Desktop/Python_SQL/Python_SQL_PI_2021_22_s2/Data/Chinook_Sqlite.sqlite"
engine = create_engine(path_to_db)
metadata_obj = MetaData() 
metadata_obj.reflect(bind=engine)
conn=engine.connect()

### Загрузим таблицы в переменные

In [5]:
album=Table("Album", metadata_obj, autoload=True, autoload_with="engine")
artist=Table("Artist", metadata_obj, autoload=True, autoload_with="engine")
customer=Table("Customer", metadata_obj, autoload=True, autoload_with="engine")
employee=Table("Employee", metadata_obj, autoload=True, autoload_with="engine")
genre=Table("Genre", metadata_obj, autoload=True, autoload_with="engine")
invoice=Table("Invoice", metadata_obj, autolad=True, autolad_with="engine")
inoviceLine=Table("InvoiceLine", metadata_obj, autolad=True, autolad_with="engine")
track=Table("Track", metadata_obj, autolad=True, autolad_with="engine")
mediatype=Table("MediaType", metadata_obj, autolad=True, autolad_with="engine")
playlist=Table("Playlist", metadata_obj, autolad=True, autolad_with="engine")
playlistTrack=Table("PlaylistTrack", metadata_obj, autolad=True, autolad_with="engine")

## 1. Выполнить отображение БД Chinook

In [25]:
for t in metadata_obj.sorted_tables:
    print(t.name)
    for c in t.columns:
        print(c)
    print()

Artist
Artist.ArtistId
Artist.Name

Employee
Employee.EmployeeId
Employee.LastName
Employee.FirstName
Employee.Title
Employee.ReportsTo
Employee.BirthDate
Employee.HireDate
Employee.Address
Employee.City
Employee.State
Employee.Country
Employee.PostalCode
Employee.Phone
Employee.Fax
Employee.Email

Genre
Genre.GenreId
Genre.Name

MediaType
MediaType.MediaTypeId
MediaType.Name

Playlist
Playlist.PlaylistId
Playlist.Name

Album
Album.AlbumId
Album.Title
Album.ArtistId

Customer
Customer.CustomerId
Customer.FirstName
Customer.LastName
Customer.Company
Customer.Address
Customer.City
Customer.State
Customer.Country
Customer.PostalCode
Customer.Phone
Customer.Fax
Customer.Email
Customer.SupportRepId

Invoice
Invoice.InvoiceId
Invoice.CustomerId
Invoice.InvoiceDate
Invoice.BillingAddress
Invoice.BillingCity
Invoice.BillingState
Invoice.BillingCountry
Invoice.BillingPostalCode
Invoice.Total

Track
Track.TrackId
Track.Name
Track.AlbumId
Track.MediaTypeId
Track.GenreId
Track.Composer
Track.Milli

## 2. Выполнить запросы с помощью подзапросов.

## 2.1. Составьте запрос, выполняющий вывод информации о счетах клиентов из Бразилии. Результат должен содержать фамилию и имя клиента, идентификатор счета, дату выставления счета, название страны плательщика (Billing Country).

In [106]:
subquery = select(customer.c.CustomerId)
subquery = subquery.where(customer.c.Country == "Brazil").scalar_subquery()

s = select(customer.c.FirstName, customer.c.LastName, invoice.c.InvoiceId, func.date(invoice.c.InvoiceDate), invoice.c.BillingCountry)
s = s.where(invoice.c.CustomerId == subquery)

# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

[('Luís', 'Gonçalves', 98, '2010-03-11', 'Brazil'), ('Leonie', 'Köhler', 98, '2010-03-11', 'Brazil'), ('François', 'Tremblay', 98, '2010-03-11', 'Brazil'), ('Bjørn', 'Hansen', 98, '2010-03-11', 'Brazil'), ('František', 'Wichterlová', 98, '2010-03-11', 'Brazil')]
Amount = 413


## 2.2. Покажите сотрудников, подчиняющихся сотруднику *Nancy Edwards*.

In [105]:
subquery = select(employee.c.EmployeeId)
subquery = subquery.where(employee.c.FirstName == "Nancy" and employee.c.LastName == "Edwards")
subquery_check = subquery
subquery = subquery.scalar_subquery()

s = select(employee.c.EmployeeId, employee.c.FirstName, employee.c.LastName, employee.c.ReportsTo)
s = s.where(employee.c.ReportsTo.in_(subquery))

# Выведем Id сотрудника Nancy Edwards 
print(conn.execute(subquery_check).fetchall())

result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

[(2,)]
[(3, 'Jane', 'Peacock', 2), (4, 'Margaret', 'Park', 2), (5, 'Steve', 'Johnson', 2)]
Amount = 3


## 2.3. Покажите трэки в жанре *Classical*, длящиеся более 8 минут.

In [6]:
subquery = select(genre.c.GenreId)
subquery = subquery.where(genre.c.Name == "Classical")
subquery_check = subquery
subquery = subquery.scalar_subquery()

s = select(track.c.TrackId, track.c.Name, track.c.Composer, track.c.GenreId, track.c.Milliseconds)
s = s.where(track.c.GenreId == subquery and track.c.Milliseconds > 8*60*10**3)

# Выведем Id жанра Classical и 8 минут в миллисекндах
print(conn.execute(subquery_check).fetchall())
print(f"8 minutes = {8*60*10**3} milliseconds")
# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

[(24,)]
8 minutes = 480000 milliseconds
[(3359, 'Symphony No. 3 in E-flat major, Op. 55, "Eroica" - Scherzo: Allegro Vivace', 'Ludwig van Beethoven', 24, 356426), (3403, 'Intoitus: Adorate Deum', 'Anonymous', 24, 245317), (3404, 'Miserere mei, Deus', 'Gregorio Allegri', 24, 501503), (3405, 'Canon and Gigue in D Major: I. Canon', 'Johann Pachelbel', 24, 271788), (3406, 'Concerto No. 1 in E Major, RV 269 "Spring": I. Allegro', 'Antonio Vivaldi', 24, 199086)]
Amount = 74


## 2.4. Покажите альбомы, в составе которых более 25 трэков.

In [103]:
subquery = select(track.c.AlbumId)
subquery = subquery.group_by(track.c.AlbumId)
subquery = subquery.having(func.count(track.c.AlbumId) > 25)
subquery_check = subquery
subquery = subquery.scalar_subquery()

# Выведем Id альбомов, где более 25 треков.
print(conn.execute(subquery_check).fetchall())


s = select(album.c.AlbumId, album.c.Title)
s = s.where(album.c.AlbumId.in_(subquery))

# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

[(23,), (73,), (141,), (229,)]
[(23, 'Minha Historia'), (73, 'Unplugged'), (141, 'Greatest Hits'), (229, 'Lost, Season 3')]
Amount = 4


### По непонятной для меня причине, если поменять последнюю строку запроса с  
### s = s.where(album.c.AlbumId.in_(subquery))
### на s = s.where(album.c.AlbumId == subquery)
### То будет отображаться только (23, 'Minha Historia'), а не 4 альбома, как сейчас.
### Если знаете, почему так происходит и почему не происходило в запросах выше и на паре, объясните по электронной почте

## 2.5. Покажите жанры, в составе которых имеются трэки, длящиеся более 18 минут.

In [102]:
subquery = select(func.distinct(track.c.GenreId))
subquery = subquery.where(track.c.Milliseconds > 18*60*10**3)
subquery_check = subquery
subquery = subquery.scalar_subquery()

s = select(genre.c.GenreId, genre.c.Name)
s = s.where(genre.c.GenreId.in_(subquery))

# Выведем Id жанров и 18 минут в миллисекндах
print(conn.execute(subquery_check).fetchall())
print(f"18 minutes = {18*60*10**3} milliseconds")
# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

[(1,), (18,), (19,), (20,), (21,), (22,)]
18 minutes = 1080000 milliseconds
[(1, 'Rock'), (18, 'Science Fiction'), (19, 'TV Shows'), (20, 'Sci Fi & Fantasy'), (21, 'Drama')]
Amount = 6


### Та же ситуация, как и с прошлым заданием

## 2.6. Покажите артистов в жанре *Classical*.

In [107]:
subquery_1 = select(genre.c.GenreId)
subquery_1 = subquery_1.where(genre.c.Name == "Classical")
subquery_check = subquery_1
subquery_1 = subquery_1.scalar_subquery()

# Выведем Id жанра Classical
print(f"Classical ID")
print(conn.execute(subquery_check).fetchall())

subquery_2 = select(func.distinct(track.c.AlbumId))
subquery_2 = subquery_2.where(track.c.GenreId.in_(subquery_1))
subquery_check = subquery_2
subquery_2 = subquery_2.scalar_subquery()
# Выведем Id альбомов в жанре Classical
print(f"Classical alboms")
print(conn.execute(subquery_check).fetchall())

subquery_3 = select(func.distinct(album.c.ArtistId))
subquery_3 = subquery_3.where(album.c.ArtistId.in_(subquery_2))
subquery_check = subquery_3
subquery_3 = subquery_3.scalar_subquery()
# Выведем Id артистов, которые имеют альбом(-ы) в жанре Classical
print(f"Artists")
print(conn.execute(subquery_check).fetchall())

s = select(artist.c.ArtistId, artist.c.Name)
s = s.where(artist.c.ArtistId.in_(subquery_3))

# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

Classical ID
[(24,)]
Classical alboms
[(268,), (272,), (273,), (274,), (275,), (276,), (277,), (278,), (279,), (280,), (281,), (282,), (283,), (284,), (285,), (286,), (287,), (288,), (289,), (290,), (291,), (292,), (293,), (294,), (295,), (296,), (297,), (298,), (299,), (300,), (301,), (302,), (303,), (304,), (305,), (306,), (307,), (308,), (309,), (310,), (311,), (312,), (313,), (314,), (315,), (316,), (318,), (319,), (320,), (324,), (325,), (326,), (327,), (328,), (329,), (330,), (331,), (332,), (333,), (334,), (335,), (336,), (337,), (338,), (339,), (340,), (341,), (342,), (343,), (344,), (345,), (346,)]
Artists
[(268,), (272,), (273,), (274,), (275,)]
[(268, 'Itzhak Perlman'), (272, 'Emerson String Quartet'), (273, 'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu'), (274, 'Nash Ensemble'), (275, 'Philip Glass Ensemble')]
Amount = 5


## 2.7. Найдите артистов, записавших альбомы, в которых более 20 произведений.

In [110]:
subquery_1 = select(track.c.AlbumId)
subquery_1 = subquery_1.group_by(track.c.AlbumId)
subquery_1 = subquery_1.having(func.count(track.c.AlbumId) > 20)
subquery_check = subquery_1
subquery_1 = subquery_1.scalar_subquery()

# Выведем Id альбомов, содержащих более 20 произведений
print(f"More than 20 tracks")
print(conn.execute(subquery_check).fetchall())

subquery_2 = select(func.distinct(album.c.ArtistId))
subquery_2 = subquery_2.where(album.c.AlbumId.in_(subquery_1))
subquery_check = subquery_2
subquery_2 = subquery_2.scalar_subquery()
# Выведем Id артистов - авторов тех самых альбомов
print(f"Artists Id's")
print(conn.execute(subquery_check).fetchall())

print(f"Artists")

s = select(artist.c.ArtistId, artist.c.Name)
s = s.where(artist.c.ArtistId.in_(subquery_2))

# print(s)
result = conn.execute(s).fetchall()
print(result[:5])
print(f"Amount = {len(result)}")

More than 20 tracks
[(23,), (24,), (39,), (51,), (73,), (83,), (141,), (167,), (224,), (228,), (229,), (230,), (231,), (250,), (251,), (253,), (255,)]
Artists Id's
[(17,), (18,), (54,), (69,), (81,), (85,), (100,), (113,), (146,), (148,), (149,), (156,), (158,), (150,)]
Artists
[(17, 'Chico Buarque'), (18, 'Chico Science & Nação Zumbi'), (54, 'Green Day'), (69, 'Gene Krupa'), (81, 'Eric Clapton')]
Amount = 14
